## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Port Alfred,-33.5906,26.8910,60.19,85,10,3.00,ZA,clear sky
1,1,Jamestown,42.0970,-79.2353,77.02,73,75,8.05,US,broken clouds
2,2,Rikitea,-23.1203,-134.9692,72.82,74,3,15.43,PF,clear sky
3,3,Hilo,19.7297,-155.0900,82.26,84,3,1.99,US,clear sky
4,4,Hithadhoo,-0.6000,73.0833,81.57,72,3,3.44,MV,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
locations = city_data_df[["Lat", "Lng"]]

# Get max temp:

max_temp= city_data_df["Max Temp"]
temps= []

for temp in max_temp:
    temps.append(max(temp, 0))
    
# Assign the figure variable to the gmaps.figure() attribute.  
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights= [max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

# 5. Add the heatmap layer.
fig.add_layer(heat_layer)

# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Ask the customer to add a minimum and maximum temp value:

min_temp=float(input("What is the minimum temperature you would like for your trip? "))
max_temp=float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                     (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
1,1,Jamestown,42.0970,-79.2353,77.02,73,75,8.05,US,broken clouds
3,3,Hilo,19.7297,-155.0900,82.26,84,3,1.99,US,clear sky
4,4,Hithadhoo,-0.6000,73.0833,81.57,72,3,3.44,MV,clear sky
7,7,Bethel,41.3712,-73.4140,78.19,62,0,5.01,US,clear sky
9,9,Aguimes,27.9054,-15.4461,77.67,69,20,17.27,ES,few clouds
10,10,Ribeira Grande,38.5167,-28.7000,75.60,83,75,6.91,PT,light intensity shower rain
11,11,Mindelo,16.8901,-24.9804,81.95,61,75,27.63,CV,broken clouds
16,16,Pasighat,28.0667,95.3333,75.56,97,100,2.55,IN,overcast clouds
20,20,Vaitape,-16.5167,-151.7500,78.55,74,6,20.62,PF,light rain
22,22,Porto Novo,6.4965,2.6036,77.29,83,95,6.38,BJ,overcast clouds


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Check the number of rows that have a NULL value:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,77.02,broken clouds,42.0970,-79.2353,
3,Hilo,US,82.26,clear sky,19.7297,-155.0900,
4,Hithadhoo,MV,81.57,clear sky,-0.6000,73.0833,
7,Bethel,US,78.19,clear sky,41.3712,-73.4140,
9,Aguimes,ES,77.67,few clouds,27.9054,-15.4461,
10,Ribeira Grande,PT,75.60,light intensity shower rain,38.5167,-28.7000,
11,Mindelo,CV,81.95,broken clouds,16.8901,-24.9804,
16,Pasighat,IN,75.56,overcast clouds,28.0667,95.3333,
20,Vaitape,PF,78.55,light rain,-16.5167,-151.7500,
22,Porto Novo,BJ,77.29,overcast clouds,6.4965,2.6036,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"] 

      # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,77.02,broken clouds,42.0970,-79.2353,
3,Hilo,US,82.26,clear sky,19.7297,-155.0900,
4,Hithadhoo,MV,81.57,clear sky,-0.6000,73.0833,
7,Bethel,US,78.19,clear sky,41.3712,-73.4140,
9,Aguimes,ES,77.67,few clouds,27.9054,-15.4461,


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))